# IS362 Final Project
Michael Hernandez

Timothy Lynch

In [126]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# For the HTML Data we used BeautifulSoup to pull the data
res = requests.get("https://www.cdc.gov/nchs/pressroom/states/newyork/newyork.htm")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')
# this builds a list of tables
df = pd.read_html(str(table))

# The final row in the table is not data I want (it's a subtitle), so I'll loop through the tables and delete that row
# Some extra cleanup - eliminated the numeric indexes from the labels using regex

for dataframe in df:
    dataframe.drop(dataframe.tail(1).index,inplace=True)
    dataframe.iloc[:,0] = dataframe.iloc[:,0].str.replace('\d+\.', '')
    dataframe.rename(columns={ dataframe.columns[0]: "Cause of Death", dataframe.columns[2]: "Rate" }, inplace=True)

# Next I want to create the data frames from the list of data frames we created.  
# I'll pull the data using their index locations

ny_death_data_2017 = df[1]
ny_death_data_2016 = df[5]
ny_death_data_2015 = df[9]
ny_death_data_2014 = df[13]

frames = [df[1], df[5], df[9], df[13]]
yearly_dr = pd.concat(frames, keys=['2017','2016','2015','2014'], sort=True)

In [128]:
ny_death_data_2017

,Cause of Death,Deaths,Rate,State Rank*,U.S. Rate**
0,Heart Disease,44092,171.2,17th,165.0
1,Cancer,34956,141.2,41st,152.5
2,Accidents,7687,35.5,49th,49.4
3,Chronic Lower Respiratory Diseases,7258,28.9,48th,40.9
4,Stroke,6264,24.6,50th,37.6
5,Flu/Pneumonia,4517,17.7,10th,14.3
6,Diabetes,4176,16.8,47th,21.5
7,Alzheimer’s disease,3521,13.2,50th,31.0
8,Hypertension,2699,10.4,11th,9.0
9,Septicemia,2296,9.1,28th,10.6


In [134]:
df = yearly_dr.groupby(['Cause of Death'])['Deaths']

Cause of Death
 Accidents                                 7687735465155945
 Alzheimer’s disease                       3521334931742639
 Cancer                                34956353683508935392
 Chronic Lower Respiratory Disease                 71096806
 Chronic Lower Respiratory Diseases                72586860
 Diabetes                                  4176403840454064
 Flu/Pneumonia                             4517451348814702
 Heart Disease                         44092440764445043116
 Hypertension                                  269924682287
 Kidney Disease                                        2207
 Septicemia                                2296254526012568
 Stroke                                    6264625862926212
Name: Deaths, dtype: object

In [123]:
ny_death_data_2015['Deaths'].dtype

dtype('O')

In [124]:
ny_death_data_2014.columns

Index(['NY Leading Causes of Death, 2014', 'Deaths', 'Rate***', 'State Rank*',
       'U.S. Rate**'],
      dtype='object')

In [125]:
ny_death_data_2014

,"NY Leading Causes of Death, 2014",Deaths,Rate***,State Rank*,U.S. Rate**
0,1. Heart Disease,43116,178.3,16th,167.0
1,2. Cancer,35392,151.8,42nd,161.2
2,3. Chronic Lower Respiratory Disease,6806,29.1,48th,36.5
3,4. Stroke,6212,26.1,49th,40.5
4,5. Accidents,5945,27.6,49th,40.5
5,6. Flu/Pneumonia,4702,19.5,8th,15.1
6,7. Diabetes,4064,17.4,45th,20.9
7,8. Alzheimer’s disease,2639,10.7,50th,25.4
8,9. Septicemia,2568,10.8,22nd (tie),10.7
9,10. Kidney Disease,2207,9.3,41st,13.2
